## Solutions for the 02_One-level-divide-and-conquer-QAOA.ipynb

In [ ]:
# Execise 1 Solution
randomlist = np.random.choice(30000,20)

sum_of_approximations = 0

for i in range(20):
   seed = int(randomlist[i])
   sum_of_approximations += nx.algorithms.approximation.one_exchange(sampleGraph2, initial_cut=None, seed=seed)[0]

average_approx = sum_of_approximations/20
print('The greedy modularity maximization algorithm gives an average approximate Max Cut value of',average_approx)

In [ ]:
# Exercise 2 Solution
layer_count =1
results = {}
new_seed_for_each_graph = 0
shots = 25000
for key in subgraph_dictionary:
    G = subgraph_dictionary[key]
    results[key] = qaoa_for_graph(G, layer_count, shots, seed=653+new_seed_for_each_graph)
    new_seed_for_each_graph+=1
    print('The max cut QAOA coloring for the subgraph',key,'is',results[key])

In [ ]:
# Exercise 3 Solution
union_cut_value = 0
union_cut_edges = []
for u, v in sampleGraph2.edges():
    if sampleGraph2.nodes[u]['color']   !=sampleGraph2.nodes[v]['color']   :
            union_cut_value+=1
            union_cut_edges.append((u,v))
print('The max cut value approximated from the subgraph colorings with no additional optimization is',union_cut_value)


In [ ]:
# Exercise 4 Solution
# Add attribute to capture the penalties of changing subgraph colors

# Initialize all the penalties to 0
nx.set_edge_attributes(mergerGraph, int(0), 'penalty')

# Compute penalties for each edge
for i, j in mergerGraph.edges():
   penalty_ij = 0
   for u in subgraph_dictionary[i].nodes():
      for neighbor_u in nx.all_neighbors(sampleGraph2, u):
         if neighbor_u in subgraph_dictionary[j]:
            if sampleGraph2.nodes[u]['color'] != sampleGraph2.nodes[neighbor_u]['color']:
               penalty_ij += 1
            else:
               penalty_ij += -1
   mergerGraph[i][j]['penalty'] = penalty_ij


# Graph the penalties of each edge
edge_labels = nx.get_edge_attributes(mergerGraph, 'penalty')
nx.draw_networkx_edge_labels(mergerGraph,  edge_labels=edge_labels,  pos=pos_merger)
nx.draw(mergerGraph, node_color = merger_color_map, pos = pos_merger, with_labels=True)
plt.show()



In [ ]:
# Exercise 5 Solution
# Run QAOA on the merger subgraph to identify which subgraphs
# if any should change colors

layer_count_merger = 1 # set arbitrarily
parameter_count_merger: int = 2 * layer_count_merger

# Specify the optimizer and its initial parameters. Make it repeatable.
cudaq.set_random_seed(101)
optimizer_merger = cudaq.optimizers.COBYLA()
np.random.seed(101)
optimizer_merger.initial_parameters = np.random.uniform(-np.pi, np.pi,
                                                     parameter_count_merger)
optimizer_merger.max_iterations=150

merger_nodes = list(mergerGraph.nodes())
qubit_count = len(merger_nodes)
merger_edge_src = []
merger_edge_tgt = []
for u, v in nx.edges(mergerGraph):
    # We can use the index() command to read out the qubits associated with the vertex u and v.
    merger_edge_src.append(merger_nodes.index(u))
    merger_edge_tgt.append(merger_nodes.index(v))

# Pass the kernel, spin operator, and optimizer to `cudaq.vqe`.
optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_qaoa,
    spin_operator=mHamiltonian(mergerGraph),
    argument_mapper=lambda parameter_vector: (qubit_count, layer_count, merger_edge_src, merger_edge_tgt, parameter_vector),
    optimizer=optimizer_merger,
    parameter_count=parameter_count_merger,
    shots = 10000)

# Print the optimized value and its parameters
print("Optimal value = ", optimal_expectation)
print("Optimal parameters = ", optimal_parameters)

# Sample the circuit using the optimized parameters
sample_number=15000
counts = cudaq.sample(kernel_qaoa, qubit_count, layer_count, merger_edge_src, merger_edge_tgt, optimal_parameters, shots_count=shots)
print(f"most_probable = {counts.most_probable()}")

# Merger results
mergerResultsString=counts.most_probable()

In [ ]:
# Exercise 6 Solution
# Assign the subgraphs to the QPUs
num_qpus = 4
number_of_subgraphs = len(sorted(subgraph_dictionary))
number_of_subgraphs_per_qpu = int(np.ceil(number_of_subgraphs/num_qpus))

keys_on_qpu ={}

for q in range(num_qpus):
    keys_on_qpu[q]=[]
    for k in range(number_of_subgraphs_per_qpu):
        if (k*num_qpus+q < number_of_subgraphs):
            key = sorted(subgraph_dictionary)[k*num_qpus+q]
            keys_on_qpu[q].append(key)
    print(keys_on_qpu[q],'=subgraph problems to be computed on processor',q)

In [ ]:
# Exercise 7 Solution
# Note that this cell block will generate an error message when executed in the Jupyter notebook.
# You will need to copy the code over to the Example-02-step-2.py file and 
# save the result as Example-02-step-2-Solution.py

# Let's introduce another MPI function that will be useful to
# iterate over all the GPUs.
size = comm.Get_size()

# Copy the results over to QPU 0 for consolidation
if rank!=0:
    comm.send(results, dest=0, tag=0)
    print("{} sent by processor {}".format(results, rank))
else:
    for j in range(1,size,1):
        colors = comm.recv(source=j, tag=0)
        print("Received {} from processor {}".format(colors, j))
        for key in colors:
            results[key]=colors[key]
    print("The results dictionary on GPU 0 =", results)
